In [1]:
import data
import utils
import time
import os

import torch # The top-level PyTorch package and tensor library.
import torch.nn as nn # A subpackage that contains modules and extensible classes for building neural networks.
import torch.optim as optim # A subpackage that contains standard optimization operations like SGD and Adam.
import torch.nn.functional as F # functional interface that contains typical operations used for building neural networks like loss functions and convolutions.

import torchvision # package that provides access to popular datasets, model architectures, and image transformations for computer vision.
import torchvision.transforms as transforms # An interface that contains common transforms for image processing.

torch.set_printoptions(linewidth=120) # to print in one line
torch.set_grad_enabled(True)#to update new gradients have to have past gradients so make pytorch to trach the grad values

import itertools
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
from torch.autograd import Variable
from torch.utils.data import DataLoader

In [2]:
train_transforms = transforms.Compose([transforms.RandomCrop(224),
                                       transforms.ToTensor()])

train_set = data.ForestData('data/check.csv', 
                           'data/train_check',
                           'data/labels.txt', 
                           train_transforms)

In [3]:
class Model(nn.Module):

    def __init__(self):
        super().__init__()
        self.pretrained_model = torchvision.models.resnet18(pretrained=True)
        classifier = [
            nn.Linear(self.pretrained_model.fc.in_features, 17)
        ]
        self.classifier = nn.Sequential(*classifier)
        self.pretrained_model.fc = self.classifier

    def forward(self, x):
        x = self.pretrained_model(x)
        return F.sigmoid(x)

In [4]:
PATH = "entire_model_resnet18.pt" 

model = Model()
model.load_state_dict(torch.load(PATH))
model.eval()

Model(
  (pretrained_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, tr

In [5]:
# disable the gradient tracking as here we are testing the data set no training happen 
@torch.no_grad()
def get_all_preds(model, loader):
    all_preds = torch.tensor([])
    all_labels = torch.tensor([])
    for batch in loader:
        images, labels = batch

        preds = model(images) # here network is the model that we usually used 
        all_preds = torch.cat(
            (all_preds, preds)
            ,dim=0
        )
        all_labels = torch.cat(
            (all_labels, labels)
            ,dim=0
        )
    return all_preds, all_labels

In [6]:
with torch.no_grad():
    prediction_loader = torch.utils.data.DataLoader(train_set, batch_size=1000)
    train_preds, labels = get_all_preds(model, prediction_loader)

C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [141]:
num = 545

In [142]:
print(train_preds[num])

tensor([8.8006e-04, 2.1146e-03, 2.1607e-03, 9.9991e-01, 1.7784e-03, 1.4889e-03, 1.4059e-03, 1.3060e-03, 2.8529e-03,
        3.3942e-03, 1.3746e-03, 1.2290e-03, 2.9386e-03, 1.8129e-03, 1.5833e-03, 5.0089e-03, 1.4125e-03])


In [143]:
#print(torchvision.transforms.Normalize(train_preds[num]))

"""mean = train_preds[num].mean()
std = train_preds[num].std()


for i in range(17):
    train_preds[num][i] = (train_preds[num][i] - mean)/std

print(train_preds[num])"""

'mean = train_preds[num].mean()\nstd = train_preds[num].std()\n\n\nfor i in range(17):\n    train_preds[num][i] = (train_preds[num][i] - mean)/std\n\nprint(train_preds[num])'

In [144]:
print(train_preds[num].mean())
sum = 0
deviations = 0

for i in range(17):
    sum = sum + train_preds[num][i]
    
sum = sum - train_preds[num][3]   
mean = sum / 16
print(mean)
for i in range(17):
    if (i != 3):
        print(i)
        deviations = deviations + ((train_preds[num][i] - mean) ** 2)
print(deviations/16)

for i in range(17):
    if (train_preds[num][i] >= mean):
        train_preds[num][i] = 1
    
    else:
        train_preds[num][i] = 0
        
print(train_preds[num])        

tensor(0.0607)
tensor(0.0020)
0
1
2
4
5
6
7
8
9
10
11
12
13
14
15
16
tensor(1.0343e-06)
tensor([0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0.])


In [145]:
print(labels[num])

tensor([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])


In [ ]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [20]:
preds_correct = get_num_correct(train_preds, labels) # observe the function above

RuntimeError: The size of tensor a (1000) must match the size of tensor b (17) at non-singleton dimension 1

In [ ]:
print('total correct:', preds_correct)
print('accuracy:', preds_correct / len(train_set))

In [ ]:
labels

In [ ]:
test_preds.argmax(dim=1)

In [ ]:
# stack these two tensors along the second dimension so can have 60,000 ordered pairs.
stacked = torch.stack(
    (
        labels
        ,train_preds.argmax(dim=1)
    )
    ,dim=1
)

stacked.shape

In [ ]:
cmt = torch.zeros(10,10, dtype=torch.int64)

for p in stacked:
    tl, pl = p.tolist()
    cmt[tl, pl] = cmt[tl, pl] + 1 
    
cm = confusion_matrix(labels, train_preds.argmax(dim=1))    

In [ ]:
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
plt.figure(figsize=(10,10))
plot_confusion_matrix(cm, train_set.classes)